# VQE experiments
Runs VQE experiments on both the ideal QASM simulator and IBM Quantum Computers for different bond distances in the H2 molecule
IBM-Q implementation trough Qiskit runtime so that error mitigation techniques can be easily activated (see `resilience_level` and documentation <https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-mitigation.html>)

Additionally, for comparison, the exact solution for the ground state energy is computed through diagonalization of the Hamiltonian.

Notebook created by **Jonas Jaeger** <jojaeger@cs.ubc.ca>

In [ ]:
from hamiltonian import MolecularFermionicHamiltonian
from mapping import JordanWigner
from pyscf import gto
from solver import ExactSolver
from solver import VQESolver
from estimator import BasicEstimator
import csv
from scipy.optimize import minimize
from qiskit import Aer
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options
from qiskit.circuit import Parameter, QuantumCircuit


options = Options()
options.resilience_level = 0
options.optimization_level = 0
service = QiskitRuntimeService(instance="ibm-q-qida/pinq2/winter-school")
session_backend = service.backends("ibmq_lima")[0]

# Define ansatz:
varform_4qubits_1param = QuantumCircuit(4)
a = Parameter('a')
varform_4qubits_1param.ry(a,0)
varform_4qubits_1param.cnot(0, 1)
varform_4qubits_1param.cnot(1, 2)
varform_4qubits_1param.cnot(2, 3)
varform_4qubits_1param.x([1, 3])

with open("dissociation_vals.csv", "a") as csv_file:
    writer = csv.writer(csv_file, delimiter=',')

    with Session(service=service, backend=session_backend) as session:
        runtime_estimator = Estimator(session=session, options=options)

        # distance unit in AA
        for distance in [0.735, 0.3, 0.5, 1., 1.5, 2.]: # or use: np.linspace(start=0.2, stop=2.5, num=100).tolist():
            mol = gto.M(
                atom = [['H', (0,0,-distance/2)], ['H', (0,0,distance/2)]],
                basis = 'sto-3g'
                )
            energy_nuc = mol.energy_nuc()
            print(distance)
            molecular_hamiltonian_no_spin = MolecularFermionicHamiltonian.from_pyscf_mol(mol)
            print(molecular_hamiltonian_no_spin)
            print(*molecular_hamiltonian_no_spin.get_integrals(), sep="\n\n")

            molecular_hamiltonian = molecular_hamiltonian_no_spin.include_spin()

            mapping = JordanWigner()
            qubit_hamiltonian = mapping.fermionic_hamiltonian_to_qubit_hamiltonian(molecular_hamiltonian).sort()
            print(qubit_hamiltonian)

            # Exact
            exact_solver = ExactSolver()
            ground_state_value = exact_solver.lowest_eig_value(qubit_hamiltonian)
            print('Ground state energy (electronic, exact) : ', ground_state_value)
            print('Ground state energy (molecular, exact) : ', ground_state_value + energy_nuc)
            exact_energy = ground_state_value + energy_nuc

            # VQE Sim
            backend = Aer.get_backend('qasm_simulator')
            minimizer = lambda fct, start_param_values: minimize(
                fct,
                start_param_values,
                method='SLSQP',
                options={'maxiter': 20, 'eps': 1e-1, 'ftol': 1e-4, 'disp': True, 'iprint': 2})
            varform = varform_4qubits_1param
            execute_opts = {'shots': 8192}
            estimator = BasicEstimator(varform, backend, execute_opts=execute_opts)
            estimator.set_observable(qubit_hamiltonian)
            vqe_solver = VQESolver(estimator, minimizer, [0, ], name='vqe_solver')
            opt_value = vqe_solver.lowest_eig_value(qubit_hamiltonian)
            print('Ground state position estimate (vqe) : ', vqe_solver.last_opt_params)
            print('Ground state energy estimate (electronic, vqe) : ', opt_value)
            print('Ground state energy estimate (molecular, vqe) : ', opt_value + energy_nuc)
            vqe_energy_sim = opt_value + energy_nuc

            # VQE IBMQ
            minimizer = lambda fct, start_param_values: minimize(
                fct,
                start_param_values,
                method='SLSQP',
                options={'maxiter': 20, 'eps': 1e-1, 'ftol': 1e-4, 'disp': True, 'iprint': 2})
            varform = varform_4qubits_1param
            execute_opts = {'shots': 8192}
            estimator = BasicEstimator(varform, runtime_estimator, execute_opts=execute_opts)
            estimator.set_observable(qubit_hamiltonian)
            vqe_solver = VQESolver(estimator, minimizer, [0, ], name='vqe_solver')
            opt_value = vqe_solver.lowest_eig_value(qubit_hamiltonian)
            print('Ground state position estimate (vqe) : ', vqe_solver.last_opt_params)
            print('Ground state energy estimate (electronic, vqe) : ', opt_value)
            print('Ground state energy estimate (molecular, vqe) : ', opt_value + energy_nuc)
            vqe_energy_ibmq = opt_value + energy_nuc

            writer.writerow([distance, exact_energy, vqe_energy_sim, vqe_energy_ibmq])